In [ ]:
!pip install datasets
!pip install openai
!pip install guidance

In [ ]:
import openai
openai.api_key = ? # YOUR_API_KEY

In [ ]:
from datasets import load_dataset
import pandas as pd
import guidance
import re
import numpy as np

In [ ]:
# Load the dataset
dataset = load_dataset("civil_comments", split="train")
df = pd.DataFrame(dataset)
ranges = [(0.0, 0.25), (0.25, 0.5), (0.5, 0.75), (0.75, 1.01)]
sampled_dfs = []
for lower, upper in ranges:
    range_df = df[(df['toxicity'] >= lower) & (df['toxicity'] < upper)]
    sampled_df = range_df.sample(n=200, random_state=42)  # set random seed for reproducibility
    sampled_dfs.append(sampled_df)
final_df = pd.concat(sampled_dfs, ignore_index=True)
print(final_df[['text', 'toxicity']])

In [ ]:
guidance.llm = guidance.llms.OpenAI(
    'gpt-4',
    caching=False
)

In [ ]:
single_program = guidance(
  '''
  {{#system}}Evaluate the toxicity score from 0 to 1 of the following sentence, provide your brief reason first and then show the toxicity score (from 0 to 1) in the final line: {{/system}}

  {{#user}}
  {{input}}
  {{/user}}
  {{#assistant}}{{gen "score" temperature=temperature max_tokens=max_tokens}}{{/assistant}}
  ''', silent=True
)

coded_program = guidance(
  '''
  {{#system}}Evaluate the toxicity score from 0 to 1 of the following four sentences, provide your brief reason and score for each one, and finally list the four scores in the last line, separated by commas: {{/system}}

  {{#user}}
  [1]
  {{input1}}

  [2]
  {{input2}}

  [3]
  {{input3}}

  [4]
  {{input4}}

  {{/user}}
  {{#assistant}}{{gen "score" temperature=temperature max_tokens=max_tokens}}{{/assistant}}
  ''', silent=True
)

In [ ]:
# dec2
def solve(y_label, y_uncoded, y_coded, L = 1):
  A = np.array([[1, 0, 0, 0],
          [0, 1, 0, 0],
          [0, 0, 1, 0],
          [0, 0, 0, 1],
          [L, -L, 0, 0],
          [L, 0, -L, 0],
          [L, 0, 0, -L],
          [0, L, -L, 0],
          [0, L, 0, -L],
          [0, 0, L, -L]])
  y_uncoded = np.array(y_uncoded)
  y_coded = np.array(y_coded)
  b = np.zeros(10)
  b[:4] = y_uncoded
  b[4:7] = y_coded[0] - y_coded[1:]
  b[7:9] = y_coded[1] - y_coded[2:]
  b[9] = y_coded[2] - y_coded[3:]
  b[4:10] = [value * L for value in b[4:10]]

  # Solve the least squares problem
  y_decoded, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
  print('Solution vector before clipping:', y_decoded)
  y_decoded = np.clip(y_decoded, 0, 1)

  print('y_decoded:', y_decoded)

  # error with uncoded prompts
  err_uncoded = np.sum((y_uncoded - y_label)**2)

  # error with one coded prompt
  err_coded = np.sum((y_coded - y_label)**2)

  # error after decoding
  err_decoded = np.sum((y_decoded - y_label)**2)

  return err_uncoded, err_coded, err_decoded

In [ ]:
# dec1
def solve_avg(y_label, y_uncoded, y_coded, L = 1):
  y_uncoded = np.array(y_uncoded)
  y_coded = np.array(y_coded)
  y_decoded = [(a + b) / 2 for a, b in zip(y_uncoded, y_coded)]
  y_decoded = np.array(y_decoded)

  print('y_decoded_avg:', y_decoded)

  # error with uncoded prompts
  err_uncoded = np.sum((y_uncoded - y_label)**2)

  # error with one coded prompt
  err_coded = np.sum((y_coded - y_label)**2)

  # error after decoding
  err_decoded = np.sum((y_decoded - y_label)**2)

  return err_uncoded, err_coded, err_decoded

In [ ]:
np.random.seed(42)
T = 100
total_err_uncoded = 0
total_err_coded = 0
total_err_decoded = 0
total_err_decoded_avg = 0

for tt in range(T):
  print('----------', tt, '----------')
  # Randomly select 4 sentences from the final_df
  samp = final_df.sample(n=4, random_state=None)

  # True Label
  y_label = []
  for i in range(4):
    y_label.append(samp.iloc[i]['toxicity'])

  # Uncoded
  y_uncoded = []
  for i in range(4):
    out = single_program(temperature=0.0, max_tokens=1024, input = samp.iloc[i]['text'])
    last_line = out['score'].strip().split('\n')[-1]
    match = re.search(r'([\d.]+)\s*$', last_line)
    if match:
        number = float(match.group(1))
        y_uncoded.append(number)
    else:
        print("No number found!")

  # coded
  out = coded_program(temperature=0.0, max_tokens=2048, input1 = samp.iloc[0]['text'], input2 = samp.iloc[1]['text'], input3 = samp.iloc[2]['text'], input4 = samp.iloc[3]['text'])
  last_line = out['score'].strip().split('\n')[-1]
  match = re.findall(r'(\d+\.\d+|0|1)', last_line)
  if match:
      y_coded = [float(value) for value in match]
  else:
      print("No number found!")

  # check
  if len(y_label) != 4 or len(y_uncoded) != 4 or len(y_coded) != 4:
    print("At least one of the lists does not contain exactly four items.")
    continue
  else:
    print("y_label:", y_label)
    print("y_uncoded:", y_uncoded)
    print("y_coded:", y_coded)

  err_uncoded, err_coded, err_decoded = solve(y_label, y_uncoded, y_coded, L = 1)
  err_uncoded, err_coded, err_decoded_avg = solve_avg(y_label, y_uncoded, y_coded, L = 1)
  print(err_uncoded)
  print(err_coded)
  print(err_decoded)
  print(err_decoded_avg)
  total_err_uncoded += err_uncoded
  total_err_coded += err_coded
  total_err_decoded += err_decoded
  total_err_decoded_avg += err_decoded_avg

avg_err_uncoded = total_err_uncoded / T
avg_err_coded = total_err_coded / T
avg_err_decoded = total_err_decoded / T
avg_err_decoded_avg = total_err_decoded_avg / T

print(f"Average err_uncoded: {avg_err_uncoded}")
print(f"Average err_coded: {avg_err_coded}")
print(f"Average err_decoded: {avg_err_decoded}")
print(f"Average err_decoded_avg: {avg_err_decoded_avg}")